# Spell Correction for Roman Urdu

## Data Reading

In [22]:
from spacy.lang.en import English
nlp = English()
tokenizer = nlp.tokenizer

In [23]:
corpus = []

#Reading From Corpus file
file1 = open('data.txt', 'r')
corpusLines = file1.readlines()

# Save the text in List
for line in corpusLines:
    if(line != '\n'):
        corpus.append(line.strip())
        
# Testing
print("Length of Corpus Sentences: ", len(corpus))
print("Sample: ", corpus[432])

Length of Corpus Sentences:  36600
Sample:  is liye mujhe jo bhi kaam acha lagta hai main zaroor kar rahi hon lekin film ka naam apne role aur film ki kahani ke barey mien kuch nahi bata sakti


In [24]:
corpusWords = []

# Tokenizing corpus into a list of words
for line in corpus:
    if(line != '\n'):
        sentence = tokenizer(line)
        for word in sentence:
            if(word.text != "\t" and word.text != "\n"):
                corpusWords.append(word.text)
                
# Testing
print("Length of Corpus Words: ", len(corpusWords))
print("Sample: ", corpusWords[0])

Length of Corpus Words:  41180
Sample:  sai


In [25]:
misspelledWords = []

#Reading From MisspelledWords file
file2 = open('misspellings.txt', 'r')
misspelledLines = file2.readlines()

# Tokenizing the words and saving in List
sentenceCounter = 0
for line in misspelledLines:
    if(line != '\n'):
        sentence = tokenizer(line)
        for index, word in enumerate(sentence):
            if(index == 0):
                misspelledWords.append([word.text])
            if(index > 1):
                if(word.text != "\t" and word.text != "\n"):
                    misspelledWords[sentenceCounter].append(word.text)
        sentenceCounter += 1

# Testing
print("Length of Misspelled Words: ", len(misspelledWords))
print("Sample:")
print(misspelledWords[0])
print(misspelledWords[1])

Length of Misspelled Words:  36101
Sample:
['Correct', 'Wrong']
['ka', 'kaz', 'cka', 'mka', 'kga', 'yka', 'kba']


# Language Models

### UNI-GRAM MODEL

In [26]:
unigrams = {}
#unigram is a dict of words corresponding to their probabitlty of existance
# Formula of UNIGRAM: P(wi) = count(wi) / count(total number of words)
totalNumberOfWords = len(corpusWords)
unigramsCount = {}
# Getting Count for each word
for word in corpusWords:
    if word not in unigramsCount:
        unigramsCount[word] = 1
    else:
        unigramsCount[word] += 1
# Counting unigram probabilities
print(totalNumberOfWords)
for word in unigramsCount:
    #print(unigramsCount[word])
    wordProbability = unigramsCount[word] / totalNumberOfWords
    unigrams[word] = wordProbability
# Testing
print("Length of UNIGRAM: ", len(unigrams))
print("Sample: ", unigrams["azad"])

41180
Length of UNIGRAM:  36044
Sample:  9.713453132588635e-05


### For Error Model

In [27]:
# Generating Bigrams

bigrams = []
unigramsForErrorModel = []
totalWords = set(corpusWords)

for index, row in enumerate(corpusWords):
    for letterNumber in range(0, len(row)-1):
        bigrams.append(row[letterNumber]+row[letterNumber+1])
        unigramsForErrorModel.append(row[letterNumber])
        if(letterNumber == len(row)-1):
            unigramsForErrorModel.append(row[letterNumber+1])

#Testing
print("Converted: ", corpusWords[4], "into:")
print(bigrams[0])
print(bigrams[1])
print(bigrams[2])
print(bigrams[3])

bigramsCount = {}
unigramsCountForErrorModel = {}

# Getting Count for each word
for word in bigrams:
    
    if word not in bigramsCount:
        bigramsCount[word] = 1
    else:
        bigramsCount[word] += 1
print(bigramsCount)
# Getting Count for each word
for word in unigramsForErrorModel:
    
    if word not in unigramsCountForErrorModel:
        unigramsCountForErrorModel[word] = 1
    else:
        unigramsCountForErrorModel[word] += 1
print(unigramsCountForErrorModel)

Converted:  kisi into:
sa
ai
kh
ha
{'sa': 1757, 'ai': 1715, 'kh': 1513, 'ha': 5117, 'ya': 1302, 'he': 1302, 'er': 1780, 'ki': 756, 'is': 1258, 'si': 811, 'ka': 1772, 'ay': 2037, 'bu': 261, 'us': 718, 'ba': 1770, 'at': 2795, 'nh': 174, 'hi': 1515, 'la': 2006, 'ak': 1323, 'in': 2516, 'ma': 2291, 'al': 2363, 'bi': 456, 'aj': 470, 'au': 416, 'ur': 860, 'ir': 769, 'rf': 82, 'ky': 88, 'bt': 69, 'wa': 1216, 'ah': 1606, 'je': 169, 'ar': 4661, 're': 1503, 'wh': 60, 'aa': 2727, 'li': 1134, 'ik': 617, 'll': 566, 'eh': 656, 'hm': 152, 'fa': 562, 'rm': 225, 'ab': 1003, 'pe': 521, 'or': 1116, 'za': 639, 'im': 430, 'mo': 482, 'ko': 380, 'id': 525, 'da': 1622, 'de': 929, 'am': 1703, 'me': 917, 'ee': 1462, 'en': 1591, 'ns': 523, 'af': 466, 'ey': 401, 'th': 875, 'ri': 1595, 'ig': 261, 'gh': 534, 'ht': 509, 'ye': 548, 'to': 698, 'oh': 222, 'hy': 187, 'bo': 388, 'ho': 1405, 'oo': 1132, 'ob': 196, 'so': 483, 'tr': 459, 'ru': 295, 'ue': 148, 'lh': 60, 'md': 43, 'du': 278, 'ul': 543, 'il': 935, 'ra': 2771, '

In [28]:
#Some Cleaning
bigrams.clear()
unigramsForErrorModel.clear()

## Pretty Tables Printings

In [29]:
import prettytable

# For Confusion Matrices
def print_confusionMatrix(confusionMatrix):

    tableCourses = prettytable.PrettyTable()
    tableCourses.field_names  = ['#','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    tableCourses._max_width = {'#':1,'a':1,'b':1,'c':1,'d':1,'e':1,'f':1,'g':1,'h':1,'i':1,'j':1,'k':1,'l':1,'m':1,'n':1,'o':1,'p':1,'q':1,'r':1,'s':1,'t':1,'u':1,'v':1,'w':1,'x':1,'y':1,'z':1}
    counter = 0
    for rowCount,row in enumerate(confusionMatrix):

        if(rowCount > 0):
            items = []
            spaces = []
            for itemCount, item in enumerate(row):
                items.append(item)
                spaces.append(" ")

            tableCourses.add_row(items)
            tableCourses.add_row(spaces)
            counter += 1

    print(tableCourses)


# For Selection Model
def print_SelectionModel(candidateWords, wrongCharacters, correctCharacters, candidateProbabilityFromErrorModel, candidateProbabilityFromLanguageModel, candidateFinalProbability):
    
    tableCourses = prettytable.PrettyTable()
    tableCourses.field_names  = ['Candidates','x|w','P(x|w)','P(w)','P(x|w)*P(w) *10^9']

    for rowCount,row in enumerate(candidateWords):
        items = []
        
        items.append(row)
        items.append(wrongCharacters[rowCount] + "|" + correctCharacters[rowCount])
        items.append(candidateProbabilityFromErrorModel[rowCount])
        items.append(candidateProbabilityFromLanguageModel[rowCount])
        items.append(candidateFinalProbability[rowCount])

        tableCourses.add_row(items)
        
    print(tableCourses)

## Generating Confusion Matrices

In [30]:
# Initializing Confusion Matrices
import copy
alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

insertionMatrix = []
deletionMatrix = []
substitutionMatrix = []
transpositionMatrix = []

# Generating 27*27 Confusion Matrices
for index in range(0,26):
    if(index == 0):
        insertionMatrix.append(["#"] + copy.deepcopy(alphabets))
        deletionMatrix.append(["#"] + copy.deepcopy(alphabets))
        substitutionMatrix.append(["#"] + copy.deepcopy(alphabets))
        transpositionMatrix.append(["#"] + copy.deepcopy(alphabets))
        
    insertionMatrix.append(copy.deepcopy(alphabets) + ["0"])
    deletionMatrix.append(copy.deepcopy(alphabets) + ["0"])
    substitutionMatrix.append(copy.deepcopy(alphabets) + ["0"])
    transpositionMatrix.append(copy.deepcopy(alphabets) + ["0"])
    
tempAlphabets = copy.deepcopy(alphabets)
for index in range(0, 26):
    insertionMatrix[index+1][0] = tempAlphabets[index]
    deletionMatrix[index+1][0] = tempAlphabets[index]
    substitutionMatrix[index+1][0] = tempAlphabets[index]
    transpositionMatrix[index+1][0] = tempAlphabets[index]

# Changing inside values to zeros for later update
for indexOfRow, row in enumerate(insertionMatrix):
    for indexOfCol, col in enumerate(row):
        
        if(indexOfRow == 0 and indexOfCol == 0):
            insertionMatrix[indexOfRow][indexOfCol] = "#"
            deletionMatrix[indexOfRow][indexOfCol] = "#"
            substitutionMatrix[indexOfRow][indexOfCol] = "#"
            transpositionMatrix[indexOfRow][indexOfCol] = "#"
        
        if(indexOfRow > 0 and indexOfCol > 0):
            insertionMatrix[indexOfRow][indexOfCol] = 0
            deletionMatrix[indexOfRow][indexOfCol] = 0
            substitutionMatrix[indexOfRow][indexOfCol] = 0
            transpositionMatrix[indexOfRow][indexOfCol] = 0
            
# Testing
for row in transpositionMatrix:
    for item in row:
        print(item, end= " ")
    print()
    
#print(alphabets)

# a b c d e f g h i j k l m n o p q r s t u v w x y z 
a 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
b 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
c 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
d 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
e 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
f 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
g 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
h 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
i 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
j 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
k 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
l 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
m 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
o 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
p 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
q 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
r 0 0 0 0 

In [31]:
# Helpers

def change_item_string(string, new_character, position):
    string = string[:position] + new_character + string[position+1:]
    return string

def add_item_string(string, new_character, position):
    string = string[:position] + new_character + string[position:]
    return string

def remove_item_string(string, position):
    string = string[:position] + string[position+1:]
    return string

In [32]:
# Filling Confusion Matrices

def find_edit_type(correctWord, wrongWord):
    
    #To store type         // 0: insertion, 1: deletion, 2: substitution, 3: Transposition
    editType = [False]*4
    
    wi = ""
    xi = ""
    
    # If both words are of equal length
    if(len(correctWord) == len(wrongWord)):
        for index in range(0, len(correctWord)):
            # If an alphabet doesn't match
            if(correctWord[index] != wrongWord[index]):
                #Now its either substituted or transposed
                # For Substitution:
                tempWord = wrongWord
                tempWord = change_item_string(tempWord, correctWord[index], index)
                if(tempWord == correctWord):
                    editType[2] = True
                    wi = correctWord[index]
                    xi = wrongWord[index]
                    break
                # For Transposition
                tempWord = wrongWord
                if(index < len(tempWord)-1):
                    tempAlphabet = tempWord[index]
                    tempWord = change_item_string(tempWord, tempWord[index+1], index)            
                    tempWord = change_item_string(tempWord, tempAlphabet, index+1)

                    if(tempWord == correctWord):
                        editType[3] = True
                        wi = wrongWord[index+1] + wrongWord[index]
                        xi = wrongWord[index] + wrongWord[index+1]
                        break
                    
    
    # If both words are of different length
    else:
        
        ## Checking for last alphabet
        
        # Check if alphabet inserted at the end
        if(wrongWord[:len(wrongWord)-1] == correctWord):
            editType[0] = True
            wi = wrongWord[len(wrongWord)-2:len(wrongWord)-1]
            xi = wrongWord[len(wrongWord)-1:len(wrongWord)]
            #print(correctWord, "misspelled as:", wrongWord, "by inserting:", xi, "after:", wi)
            return False, editType, wi, xi

        # Check if alphabet deleted at the end
        elif(correctWord[:len(correctWord)-1] == wrongWord):
            editType[1] = True
            wi = correctWord[len(correctWord)-2:len(correctWord)-1]
            xi = correctWord[len(correctWord)-1:len(correctWord)]
            #print(correctWord, "misspelled as:", wrongWord, "by deleting:", xi, "after:", wi)
            return False, editType, wi, xi
        
        ## Checking before last alphabet
        else:
            shorterWordLength = min(len(correctWord), len(wrongWord))
            
            for index in range(0, shorterWordLength):

                # If an alphabet doesn't match
                if(correctWord[index] != wrongWord[index]):

                    # For Deletion
                    tempWord = wrongWord
                    tempWord = add_item_string(tempWord, correctWord[index], index)

                    if(tempWord == correctWord):
                        editType[1] = True
                        xi = correctWord[index]
                        wi = wrongWord[index]
                        break
                        
                    # For Insertion
                    tempWord = wrongWord
                    tempWord = remove_item_string(tempWord, index)

                    if(tempWord == correctWord):
                        editType[0] = True
                        xi = wrongWord[index]
                        wi = correctWord[index]
                        break

    return True, editType, wi, xi
print (find_edit_type("hello", "helpo"))

(True, [False, False, True, False], 'l', 'p')


In [33]:
# Updating tables for "misspellings.txt"

# Here: Wrong word on Y-AXIS & Correct word on X-AXIS
indexes = {'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,'h':8,'i':9,'j':10,'k':11,'l':12,'m':13,'n':14,'o':15,'p':16,'q':17,'r':18,'s':19,'t':20,'u':21,'v':22,'w':23,'x':24,'y':25,'z':26}

for index, row in enumerate(misspelledWords):
    # First row is: "Correct, Wrong" so skipping it!
    if(index == 0):
        continue
    for wordNumber, word in enumerate(row):
        if(wordNumber > 0 and word != ","):
            # Check for error
            errorBeforeEnd, editType, wi, xi = find_edit_type(row[0], word)
            if(errorBeforeEnd):
                #Insertion of xi before wi
                if(editType[0]):
                    Y_AXIS = indexes[wi]
                    X_AXIS = indexes[xi]
                    insertionMatrix[X_AXIS][Y_AXIS] += 1
                #Deletion of xi before wi
                elif(editType[1]):
                    Y_AXIS = indexes[wi]
                    X_AXIS = indexes[xi]
                    deletionMatrix[X_AXIS][Y_AXIS] += 1
                #Substitution of xi instead of wi
                elif(editType[2]):
                    Y_AXIS = indexes[wi]
                    X_AXIS = indexes[xi]
                    substitutionMatrix[X_AXIS][Y_AXIS] += 1
                #Transposition of xi with wi
                elif(editType[3]):
                    Y_AXIS = indexes[wi[0]]
                    X_AXIS = indexes[xi[0]]
                    transpositionMatrix[X_AXIS][Y_AXIS] += 1
            else:
                #Insertion of xi after wi
                if(editType[0]):
                    Y_AXIS = indexes[wi]
                    X_AXIS = indexes[xi]
                    insertionMatrix[X_AXIS][Y_AXIS] += 1
                #Deletion of xi after wi
                elif(editType[1]):
                    Y_AXIS = indexes[wi]
                    X_AXIS = indexes[xi]
                    deletionMatrix[X_AXIS][Y_AXIS] += 1

In [34]:
#Testing a confusion Matrix
print_confusionMatrix(substitutionMatrix)
print_confusionMatrix(insertionMatrix)

print_confusionMatrix(transpositionMatrix)

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| # | a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z |
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| a | 0 | 7 | 6 | 1 | 1 | 2 | 4 | 1 | 1 | 3 | 8 | 1 | 9 | 2 | 1 | 5 | 1 | 1 | 1 | 1 | 9 | 1 | 2 | 1 | 7 | 2 |
|   |   | 8 | 6 | 1 | 8 | 8 | 8 | 8 | 9 | 2 | 1 | 3 | 2 | 0 | 2 | 9 | 5 | 9 | 4 | 5 | 9 | 7 | 9 |   | 3 | 6 |
|   |   |   |   | 9 | 4 |   |   | 9 | 0 |   |   | 8 |   | 1 | 9 |   |   | 3 | 5 | 9 |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
| b | 6 | 0 | 6 | 1 | 2 | 2 | 6 | 1 | 2 | 2 | 8 | 1 | 8 | 2 | 2 | 6 | 1 | 2 | 1 | 1 | 1 | 2 | 2 | 5 | 8 | 3 |
|   | 4 |   | 5 | 1 | 4 | 4 | 1 | 9 | 5 | 5 | 8 | 3 | 9 | 3 | 0 | 0 | 5 | 1 | 7 | 8 | 0 | 3 | 8 |   | 5 | 1 |
|   | 2 | 

## Error Model P(x|w)

In [35]:
def error_model(correctWord, wrongWord):
    
    errorBeforeEnd, editType, wi, xi = find_edit_type(correctWord, wrongWord)
       
    #Insertion of xi before wi
    if(editType[0]):

        # P(x|w) = ins(xi-1, wi) / count(wi-1)
        indexOfInsertedLetter = wrongWord.index(xi) #xi
        indexOfInsertedAfter = indexOfInsertedLetter - 1 #wi-1

        Y_AXIS = indexes[correctWord[indexOfInsertedAfter].lower()]
        X_AXIS = indexes[wrongWord[indexOfInsertedLetter].lower()]

        countWord = unigramsCountForErrorModel[correctWord[indexOfInsertedAfter]]

        if(countWord == 0):
            return (insertionMatrix[X_AXIS][Y_AXIS] / 0 + len(corpusWords)), xi, "#"  
        else:  
            return (insertionMatrix[X_AXIS][Y_AXIS] / countWord), xi, "#"           
            
    #Deletion of xi before wi
    elif(editType[1]):
            
        # P(x|w) = del(xi-1, wi) / count(xi-1 wi)
        indexOfDeletedLetter = correctWord.index(xi) #wi
        indexOfDeletedAfter = indexOfDeletedLetter - 1 #wi-1
        
        Y_AXIS = indexes[correctWord[indexOfDeletedAfter].lower()]
        X_AXIS = indexes[correctWord[indexOfDeletedLetter].lower()]
 
        countWord = bigramsCount[correctWord[indexOfDeletedAfter] + correctWord[indexOfDeletedLetter]]

        if(countWord == 0):
            return (deletionMatrix[X_AXIS][Y_AXIS] / 0 + len(corpusWords)), "#", xi  
        else:  
            return (deletionMatrix[X_AXIS][Y_AXIS] / countWord), "#", xi  
                    
    #Substitution of xi instead of wi
    elif(editType[2]):
            
        # P(x|w) = sub(xi, wi) / count(wi)
        indexOfSubstitutedLetter = wrongWord.index(xi) #xi
        indexOfSubstitutedAfter = correctWord.index(wi) #wi
        
        Y_AXIS = indexes[correctWord[indexOfSubstitutedAfter].lower()]
        X_AXIS = indexes[wrongWord[indexOfSubstitutedLetter].lower()]
 
        countWord = unigramsCountForErrorModel[correctWord[indexOfSubstitutedAfter]]

        if(countWord == 0):
            return (substitutionMatrix[X_AXIS][Y_AXIS] / 0 + len(corpusWords)), xi, wi  
        else:  
            return (substitutionMatrix[X_AXIS][Y_AXIS] / countWord), xi, wi                     
                
                    
    #Transposition of xi with wi
    elif(editType[3]):
            
        # P(x|w) = sub(wi, wi+1) / count(wi wi+1)
        indexOfTransposedLetter = correctWord.index(wi) #wi
        indexOfTransposedBefore = indexOfTransposedLetter + 1 #wi+1
        
        Y_AXIS = indexes[correctWord[indexOfTransposedBefore].lower()]
        X_AXIS = indexes[correctWord[indexOfTransposedLetter].lower()]
 
        countWord = bigramsCount[correctWord[indexOfTransposedLetter] + correctWord[indexOfTransposedBefore]]

        if(countWord == 0):
            return (transpositionMatrix[X_AXIS][Y_AXIS] / 0 + len(corpusWords)), xi, wi  
        else:  
            return (transpositionMatrix[X_AXIS][Y_AXIS] / countWord), xi, wi                
                  

In [36]:
print(error_model("huny", "huxy"))

(0.02144286667302011, 'x', 'n')


## Generating Candidate Words

In [37]:
def edit_distance(correctWord, wrongWord):
    
    # If both words are of equal length
    if(len(correctWord) == len(wrongWord)):
        
        for index in range(0, len(correctWord)):
            
            # If an alphabet doesn't match
            if(correctWord[index] != wrongWord[index]):
                
                #Now its either substituted or transposed
                
                # For Substitution:
                tempWord = wrongWord
                
                tempWord = change_item_string(tempWord, correctWord[index], index)
                
                if(tempWord == correctWord):
                    return 1
                    
                # For Transposition
                tempWord = wrongWord
                if(index < len(tempWord)-1):
                    tempAlphabet = tempWord[index]
                    tempWord = change_item_string(tempWord, tempWord[index+1], index)            
                    tempWord = change_item_string(tempWord, tempAlphabet, index+1)

                    if(tempWord == correctWord):
                        return 1
                    
    
    # If both words are of different length
    else:
        
        ## Checking for last alphabet
        
        # Check if alphabet inserted at the end
        if(wrongWord[:len(wrongWord)-1] == correctWord):
            return 1

        # Check if alphabet deleted at the end
        elif(correctWord[:len(correctWord)-1] == wrongWord):
            return 1
        
        ## Checking before last alphabet
        else:
            shorterWordLength = min(len(correctWord), len(wrongWord))
            
            for index in range(0, shorterWordLength):

                # If an alphabet doesn't match
                if(correctWord[index] != wrongWord[index]):

                    # For Deletion
                    tempWord = wrongWord
                    tempWord = add_item_string(tempWord, correctWord[index], index)

                    if(tempWord == correctWord):
                        return 1
                        
                    # For Insertion
                    tempWord = wrongWord
                    tempWord = remove_item_string(tempWord, index)

                    if(tempWord == correctWord):
                        return 1
    
    return 0

In [38]:
def FindCandidateWords(wrongWord):
    
    candidateWords = []
    
    # Checking for 1 edit distance words in vocabulary
    for word in totalWords:
        
        editDistances = edit_distance(word, wrongWord)
        if(editDistances == 1 ):
            candidateWords.append(word)
            
    return candidateWords

## Selection Model

In [39]:
def all_possible_correct_word(wrongWord):
    
    candidateWords = FindCandidateWords(wrongWord)
    
    candidateProbabilityFromLanguageModel = [] #p(w)
    candidateProbabilityFromErrorModel = [] #p(x|w)
    candidateFinalProbability = []
    
    wrongCharacters = []
    correctCharacters = []
    
    for correctWord in candidateWords:
        probability, wrongChar, correctChar = error_model(correctWord, wrongWord)
            
        candidateProbabilityFromErrorModel.append(probability)
        wrongCharacters.append(wrongChar)
        correctCharacters.append(correctChar)
        candidateProbabilityFromLanguageModel.append(unigrams[correctWord])
        
        candidateFinalProbability.append((probability * unigrams[correctWord]) * pow(10, 9))

    print_SelectionModel(candidateWords, wrongCharacters, correctCharacters, candidateProbabilityFromErrorModel, candidateProbabilityFromLanguageModel, candidateFinalProbability)
    
    return candidateWords, candidateFinalProbability

In [40]:
def best_possible_word(wrongWord):
    print("\n\n" + "---------------------------------------------------"+"\n")
    print("Do you think " + wrongWord + " is correct [0/1] ?")
    flag1 = input("Do you think " + wrongWord + " is correct [0/1] ?")
    print(">>>>> " + flag1 +"\n")
    if(flag1 == '1'):
        with open("data.txt", "a") as a_file:
              a_file.write(wrongWord + " ")
        print(wrongWord + " is added to the dictionary" )
        return wrongWord
    elif(flag1 == '0'):
        print("What do you want, automatic or manual selection of correct word for " + wrongWord +" [A/M] ?")
        flag2 = input("What do you want, automatic or manual selection of correct word for " + wrongWord +" [A/M] ?")
        if(flag2 == 'A' or flag2 == 'a'):
            print(">>>>> Automatic \n")
            print("Candidate words for " + wrongWord + ": \n")
            candidateWords, candidateFinalProbability = all_possible_correct_word(wrongWord)
            if(candidateFinalProbability):
                max_value = max(candidateFinalProbability)
            else:
                print("We didn't find any candidate word for '" + wrongWord + "'. Please enter a word you want to replace with!! Note: This word will also be added to dictionary")
                alternate = input("We didn't find any candidate word for '" + wrongWord + "'. Please enter a word you want to replace with!! Note: This word will also be added to dictionary")
                print(">>>>> " + alternate + "\n")
                with open("data.txt", "a") as a_file:
                    a_file.write(alternate + " ")
                print(wrongWord + " is added to the dictionary and all the occurrences of '" + wrongWord + "' is replaced by '" + alternate  + "'")
                return alternate
            candidateWord = candidateWords[candidateFinalProbability.index(max_value)]
            print("Automatic Best Correction for:", wrongWord, "ïs:", candidateWord)
            return candidateWord
        elif(flag2 == 'M' or flag2 == 'm'):
            print(">>>>> Manual \n")
            print("Candidate words for " + wrongWord + ": \n")
            candidateWords, candidateFinalProbability = all_possible_correct_word(wrongWord)
            if(candidateFinalProbability):
                selectCand = input("""Select a candidate word from this list: """+  "; ".join(map(str, candidateWords)) + """!!!""")
                while(selectCand not in candidateWords):
                    selectCand = input("""You must Select a candidate word from this list: """+ "; '".join(map(str, candidateWords)) + """!!!""")
                print("Manual Correction for:", wrongWord, "ïs:", selectCand)
                return selectCand
            else:
                print("We didn't find any candidate word for '" + wrongWord + "'. Please enter a word you want to replace with!! Note: This word will also be added to dictionary")
                alternate = input("We didn't find any candidate word for " + wrongWord + ". Please enter a word you want to replace with!! Note: This word will also be added to dictionary")
                print(">>>>> " + alternate + "\n")
                with open("data.txt", "a") as a_file:
                    a_file.write(alternate + " ")
                print(wrongWord + " is added to the dictionary and all the occurrences of " + wrongWord + " is replaced by " + alternate )
                return alternate
    return wrongWord

## RESULTS

In [41]:
def search_str(file_path, word):
    with open(file_path, 'r') as file:
        content = file.read()
        if word in content:
            return True
        else:
            return False

with open('file.txt', 'r') as file:
    lfiledata = file.read()
lfiledata = lfiledata.lower()
with open('file.txt', 'w') as file:
    file.write(lfiledata)
with open('wrong.txt', 'w') as file1:
    file1.write("")
with open('file.txt', 'r') as file:
    filedata = file.read()

WordsArray = filedata.split()
mode = 'w'
for i in WordsArray:
    if (search_str('data.txt', i) == False):
        with open('wrong.txt', mode) as file:
            file.write(i + " ")
            mode = 'a'

In [42]:

with open('file.txt', 'r') as file :
  filedata = file.read()
print("Input file Text:")
print(filedata)
print("\n\n" + "---------------------------------------------------"+"\n")
with open('wrong.txt', 'r') as wfile :
  wrongdata = wfile.read()
wrongWordsArray = wrongdata.split()
print("Misspelled Words are :")
for i in wrongWordsArray:
    print(i)
for wrongWord in wrongWordsArray:
    bestWord = best_possible_word(wrongWord)
    # Replace the target string
    if (bestWord != "None"):
        filedata = filedata.replace(wrongWord, bestWord)
    # Write the file out again
    with open('file.txt', 'w') as file:
      file.write(filedata)
with open('file.txt', 'r') as file :
  filedata = file.read()
print("\n\n" + "---------------------------------------------------"+"\n")
print("Output file Text after Correction:")
print(filedata)
print("\n\n" + "---------------------------------------------------"+"\n")

Input file Text:
aj me subhaa jldai uti test tiyar kia namaz prhe pir tyar hoi or university a gai


---------------------------------------------------

Misspelled Words are :
subhaa
jldai
prhe


---------------------------------------------------

Do you think subhaa is correct [0/1] ?
>>>>> 0

What do you want, automatic or manual selection of correct word for subhaa [A/M] ?
>>>>> Manual 

Candidate words for subhaa: 

+------------+-----+----------------------+------------------------+--------------------+
| Candidates | x|w |        P(x|w)        |          P(w)          | P(x|w)*P(w) *10^9  |
+------------+-----+----------------------+------------------------+--------------------+
|   subhan   | a|n | 0.019155627561231296 | 2.4283632831471587e-05 | 465.16822635335825 |
|   subha    | a|# | 0.017793594306049824 | 4.856726566294317e-05  | 864.1862217605548  |
+------------+-----+----------------------+------------------------+--------------------+
Manual Correction for: subhaa ïs: 